<a href="https://colab.research.google.com/github/VKapicyn/m06-006sk-course-work-var2/blob/master/Ant_Colony_on_1024Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Эксперименты с графом

In [5]:
import networkx as nx

In [6]:
G = nx.gnm_random_graph(1024, 4000, directed=False)

In [50]:
sum = 0
zeros = 0
for i in G:
  sum += G.degree[i]
  if G.degree[i] == 0:
    zeros += 1
print(sum/G.number_of_nodes())
print(zeros)

7.8125
0


In [ ]:
k = 0
for i in range(0,1024):
  if nx.has_path(G,6,i):
    k += 1
print(k)

1024


# Эксперименты с муравьями

Следующий блок кода является модифицированным кодом из репозитория https://github.com/Akavall/AntColonyOptimization

---


Этот блок кода реализует прохождение муравьиной колонии по графу с целью поиска кратчайшего пути (ещё в доработке)

Алгоритм состоит из N итераций. Кол-во итераций задаётся в параметр n_iterations.

---
Содержание каждой итерации:

1. Выпускается K муравьёв (задаётся параметром n_ants) в функции **gen_all_paths()**
2. Собираются путь каждого муравья (ф-я **gen_path()**) при том, что муравей выбирает, куда ему идти в функции **pick_move()**
3. Для каждого собранного пути считается его дистанция (ф-я **gen_path_dist()**)
4. В зависимости от длинны дистанции по неё распыляется феромон (ф-я **spread_pheromone()**)




In [80]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

  def __init__(self, graph, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):

    self.graph = graph
    self.distances  = nx.to_numpy_array(graph)
    self.pheromone = np.copy(self.distances)
    self.n_ants = n_ants
    self.n_best = n_best
    self.n_iterations = n_iterations
    self.decay = decay
    self.alpha = alpha
    self.beta = beta
    self.max_moves = 3000
  #ГОТОВО
  def run(self):
    shortest_path = None
    all_time_shortest_path = ("placeholder", np.inf)
    for i in range(self.n_iterations):
        all_paths = self.gen_all_paths() #получаем все пути муравьёв за эту итерацию
        self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path) #распыляем феромоны на путях в зависимости от их итоговой длинны
        shortest_path = min(all_paths, key=lambda x: x[1]) #находим кратчайший путь из всех муравьиных
        sorted_paths = sorted(all_paths, key=lambda x: x[1]) #сортируем пути по дистанции от минимальной к максимальной, чтобы напечатать
        pr = []
        for p in sorted_paths: 
          pr.append(p[1])
        print ("short:",shortest_path[1], "pathes:", pr)
        #если нашли более короткий путь, чем был раньше, то заменяем
        if shortest_path[1] < all_time_shortest_path[1]:
            all_time_shortest_path = shortest_path            
        self.pheromone = self.pheromone * self.decay #испаряем феромон
    return all_time_shortest_path

  #ГОТОВО
  def spread_pheronome(self, all_paths, n_best, shortest_path):
    sorted_paths = sorted(all_paths, key=lambda x: x[1]) #сортируем пути по дистанции от минимальной к максимальной
    for path, dist in sorted_paths[:n_best]: #распыляем феромон на пути с наименьшей дистанцией
      if dist < self.max_moves:
        for move in path:
            self.pheromone[move[0]][move[1]] += 50.0 / dist #self.distances[move] #к феромону прибавляется значение обратно пропорциональное пройденной дистанции
  
  #ГОТОВО: суммируем длинну пути
  def gen_path_dist(self, path):
    total_dist = 0
    for step in path:
        total_dist += self.distances[step]
    return total_dist

  #ГОТОВО: собираем пути всех муравьёв
  def gen_all_paths(self):
    all_paths = []
    # выпускаем N муравьёв
    for i in range(self.n_ants):
        path = self.gen_path(start = 0) # каждый муравей проходит свой путь, начиная с 0-го узла
        all_paths.append((path, self.gen_path_dist(path))) # этот путь присоединяется к копилке всех путей этой итерации муравейника
    return all_paths

  # то, как муравей проходит свой путь
  def gen_path(self, start):
    path = []
    #visited = set()
    #visited.add(start)
    prev = start
    i = 0
    while (prev != 1023): # цикл до попадания муравья в нужный узел
      move = self.pick_move(self, self.pheromone, prev, path) # муравей выбирает следующий шаг
      path.append((prev,move))
      # муравей, зашедший в тупик, возвращается на базу до следующего выхода
      #if G.degree[move] == 1:
      #  return 0;
      # муравей, гуляющий слишком долго, возвращается на базу до следующего выхода
      i += 1
      if i >= self.max_moves:
        break
      prev = move
    return path
  

  def pick_move(self, pheromone, dist, prev, path):
      #pheromone = np.copy(pheromone)
      neighbours = np.array(list(self.graph.adj[prev])) # список соседей (вариантов)
      buff = []
      for i in neighbours:
        #if any(p[0] == i for p in path):
        if len(path) != 0 and i == path[-1][0]:
           buff.append(0.000001) # есои муравей там уже был, то вернётся с меньшей вероятностью
        else:
           buff.append(self.pheromone[prev][i])
        
      phers = np.array(buff)# список феромонов на соседях

      #for i in phers:  
      #  i = i ** self.alpha  # предпочтительность соседа при выборе пути. Забил на длинну, так как везде 1
      row = phers ** self.alpha #* (( 1.0 / dist[prev]) ** self.beta) 
      
      #row = np.ones(np.shape(neighbours))
      norm_row = row / row.sum() # нормированная предпочтительность
      
      np.random.seed()
      move = np_choice(a = neighbours, size = 1, p=norm_row)[0] # волшебство выбора
      #print(prev, " - ", move)
      return move

In [81]:
ants = AntColony(G, 25, 5, 20, 0.95, alpha=1, beta=1)
shortest_path = ants.run()

short: 131.0 pathes: [131.0, 400.0, 440.0, 529.0, 619.0, 645.0, 684.0, 694.0, 758.0, 801.0, 1003.0, 1119.0, 1131.0, 1257.0, 1318.0, 1546.0, 1617.0, 1875.0, 2079.0, 2309.0, 3000.0, 3000.0, 3000.0, 3000.0, 3000.0]
short: 78.0 pathes: [78.0, 81.0, 84.0, 141.0, 225.0, 225.0, 309.0, 322.0, 345.0, 535.0, 716.0, 748.0, 758.0, 911.0, 931.0, 1012.0, 1037.0, 1086.0, 1128.0, 1227.0, 1458.0, 2179.0, 2450.0, 2702.0, 3000.0]
short: 23.0 pathes: [23.0, 31.0, 37.0, 38.0, 160.0, 188.0, 190.0, 288.0, 353.0, 387.0, 404.0, 431.0, 464.0, 466.0, 470.0, 756.0, 951.0, 1239.0, 1382.0, 1529.0, 1968.0, 2139.0, 2751.0, 2861.0, 3000.0]
short: 50.0 pathes: [50.0, 52.0, 89.0, 216.0, 317.0, 365.0, 373.0, 383.0, 403.0, 434.0, 596.0, 643.0, 698.0, 736.0, 754.0, 853.0, 914.0, 1016.0, 1172.0, 1176.0, 1210.0, 1631.0, 1702.0, 1783.0, 1817.0]
short: 39.0 pathes: [39.0, 80.0, 86.0, 116.0, 121.0, 158.0, 163.0, 167.0, 176.0, 201.0, 235.0, 244.0, 291.0, 310.0, 331.0, 335.0, 394.0, 428.0, 430.0, 470.0, 492.0, 516.0, 758.0, 1141.

In [82]:
print(nx.dijkstra_path(G,0,1023))

[0, 465, 142, 525, 1023]
